This notebook fixes the production of ethanol in the US.

In default ecoinvent 3.4 cutoff, the supply goes like this:
    - 'dewatering of ethanol from biomass, from 95% to 99.7% solution state' (kilogram, US, None)
    - 'ethanol production from maize' (kilogram, US, None)
    - 'market for maize grain' (kilogram, GLO, None)
        - 0.396783400122377 kilogram 'maize grain production' (kilogram, US, None)
        - 0.572106021842784 kilogram 'maize grain production' (kilogram, GLO, None)

However, in our case study this doesn't make sense - we want the maize to be processed into ethanol in the US to be grown in the US. So, we link `'ethanol production from maize' (kilogram, US, None)` directly to US maize grain production.

In [ ]:
import bw2data as bd

In [ ]:
bd.projects.set_current("LC IMPACT case study")

The broken market

In [ ]:
qs = [x for x in bd.Database("ecoinvent") 
      if x['name'] == 'ethanol production from maize'
      and x['reference product'] == 'ethanol, without water, in 95% solution state, from fermentation'
      and x['location'] == 'US']

assert len(qs) == 1
ethanol_from_maize = qs[0]
ethanol_from_maize

The input of corn from the global market

In [ ]:
for fixme in ethanol_from_maize.technosphere():
    if fixme.input['name'] == 'market for maize grain':
        print(fixme)
        break

We can find the correct input (US production of corn):

In [ ]:
for usa in fixme.input.technosphere():
    if usa.input['location'] == 'US':
        print(usa)
        break

In [ ]:
fixme.input = usa.input
fixme.save()

Check to make sure it worked

In [ ]:
reloaded = bd.get_activity(ethanol_from_maize.key)
for exc in reloaded.technosphere():
    print(exc)